**Author: Muhammad**

In [ ]:
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame
import geopandas as gpd
from typing import List

/home/jovyan/venv/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


**Data Source:** Census Bureau America Community Survery 2018, 5 Year Estimates, [hosted on Google BigQuery](https://console.cloud.google.com/marketplace/product/united-states-census-bureau/acs?filter=solution-type:dataset&project=covid-disparities&organizationId=0)

In [ ]:
acs = pd.read_csv("/work/COVIDRedlining/data/ACS/acs_census_tract_2018_5yr.csv")
acs.drop(columns=["Unnamed: 0", "do_date"], inplace=True)


# Data Cleaning and Transformation for ACS Variables

Below is the list of all features original found in the ACS data set.

In [ ]:
# List of all columns
list(acs.columns)

['geo_id',
 'total_pop',
 'households',
 'male_pop',
 'female_pop',
 'median_age',
 'male_under_5',
 'male_5_to_9',
 'male_10_to_14',
 'male_15_to_17',
 'male_18_to_19',
 'male_20',
 'male_21',
 'male_22_to_24',
 'male_25_to_29',
 'male_30_to_34',
 'male_35_to_39',
 'male_40_to_44',
 'male_45_to_49',
 'male_50_to_54',
 'male_55_to_59',
 'male_60_to_61',
 'male_62_to_64',
 'male_65_to_66',
 'male_67_to_69',
 'male_70_to_74',
 'male_75_to_79',
 'male_80_to_84',
 'male_85_and_over',
 'female_under_5',
 'female_5_to_9',
 'female_10_to_14',
 'female_15_to_17',
 'female_18_to_19',
 'female_20',
 'female_21',
 'female_22_to_24',
 'female_25_to_29',
 'female_30_to_34',
 'female_35_to_39',
 'female_40_to_44',
 'female_45_to_49',
 'female_50_to_54',
 'female_55_to_59',
 'female_60_to_61',
 'female_62_to_64',
 'female_65_to_66',
 'female_67_to_69',
 'female_70_to_74',
 'female_75_to_79',
 'female_80_to_84',
 'female_85_and_over',
 'white_pop',
 'population_1_year_and_over',
 'population_3_years_o

## New Features

In [ ]:
# Other Races in the Original Data is overriden by other_race_pop = total_pop - sum(races),
# where races is shown below.
races = [
    'black_pop',
    'asian_pop',
    'hispanic_pop',
    'amerindian_pop',
    'white_pop'
]
acs['other_race_pop'] = acs.total_pop - acs[races].sum(axis=1)

# Non-white population
acs['non_white_pop'] = acs['total_pop'] - acs['white_pop']

## Data Cleaning — Explanation

In [ ]:
## General formulae:

# to_leave = ("leave", list_of_columns_to_leave_unmodified)
# to_normalise = ("norm", feature_list, normalise_by)
# to_aggregate = ("agg", feature_list, new_feature_name, normalise_by)
# to_drop = ("drop", list_of_columns_to_drop)

## Data Cleaning — Functions

In [ ]:
def aggregate_features(
    df: DataFrame, feature_list: List[str], new_feature_name:str, normalise_by:str, 
    columns_to_drop: set, suffix: str="normalised"
    ) -> None:
    """
    - Aggregates (sum) multiple columns in a given data frame
    - gives this new column a name
    - normalises the new column by some other variable
    - adds a suffix to the name of the normalised sum column
    - adds the original list of features variables as well as their unnormalised sum to the set of columns_to_drop
    """
    df[new_feature_name] = df[feature_list].sum(axis=1)
    normalise_feature(df, new_feature_name, normalise_by, suffix)
    columns_to_drop.add(new_feature_name)
    columns_to_drop.update( set(feature_list) )



def normalise_feature(df: DataFrame, feature: str, normalise_by: str, suffix: str) -> None:
    """
    Normalises a given column of the given data frame by another variable, adds a suffix to the column name, 
    and then deletes the origingal column. Then,returns this modified data frame.
    """
    df[feature + '_' + suffix] = df[feature] / df[normalise_by]
    # df.drop(columns=feature, inplace=True)


def normalise_list_of_features(
    df: DataFrame, feature_list: List[str], normalise_by: str, columns_to_drop: set, suffix: str="normalised"
    ) -> None:
    """
    Loops over a list of features in a data frame and normalises them all using the same variables 
    Adds a suffix to the column names, deletes the original columns, and returns the modified data frame.
    """
    for feature in feature_list:
        normalise_feature(df, feature, normalise_by, suffix)
        columns_to_drop.add(feature)

def clean_data(acs: DataFrame, columns_to_clean: List[tuple], columns_to_drop: set) -> DataFrame:
    df = acs.copy()
    for group in columns_to_clean:
        operation = group[0]
        try:
            if operation == 'leave':
                continue

            elif operation == 'norm':
                _, feature_list, normalise_by = group
                normalise_list_of_features(df, feature_list, normalise_by, columns_to_drop)

            elif operation == 'agg':
                _, feature_list, new_feature_name, normalise_by = group
                aggregate_features(df, feature_list, new_feature_name, normalise_by, columns_to_drop)
            
            elif operation == 'drop':
                _, feature_list = group
                columns_to_drop.update( set(feature_list) )
        except Exception:
            print("Error! The following group has an error")
            print(group)
            raise ValueError

    df.drop(columns=list(columns_to_drop), inplace=True)
    return df

### Normalise Features Test

In [ ]:
# columns_to_drop = set()
# test =  [
#             'renter_occupied_housing_units_paying_cash_median_gross_rent',
#             'owner_occupied_housing_units_lower_value_quartile', 
#             'owner_occupied_housing_units_median_value',
#             'owner_occupied_housing_units_upper_value_quartile'
#         ]

# normalise_list_of_features(acs, test, "total_pop", columns_to_drop)
# acs[test]

In [ ]:
# acs[ [x + "_normalised" for x in test] ]

### Aggregate Features Test

In [ ]:
# df = acs.copy()
# feature_list =  [
#             'female_65_to_66', 'female_67_to_69', 'female_70_to_74', 'female_75_to_79',
#             'female_80_to_84', 'female_85_and_over', 'male_65_to_66', 'male_67_to_69', 
#             'male_70_to_74', 'male_75_to_79', 'male_80_to_84', 'male_85_and_over'
# ]
# columns_to_drop = set()
# columns_to_drop.add("cars")

# aggregate_features(df, feature_list, "65_over", "total_pop", columns_to_drop)
# print(columns_to_drop)

## Data Cleaning — Sorting the Features

### Columns To Drop

The list below contains all the variables that we wish to drop immediately, without any consideration.

In [ ]:
columns_to_drop = [
    ## Already accounted for in pop cleaning above
    'hispanic_any_race', 'not_hispanic_pop',

    ## Same-sex households? 
    'male_male_households', 'female_female_households',

    ## Breakdown of pop by sexes
    'male_under_5', 'male_5_to_9', 'male_10_to_14', 'male_15_to_17', 'male_18_to_19', 'male_20',
    'male_21', 'male_22_to_24', 'male_25_to_29', 'male_30_to_34', 'male_35_to_39', 'male_40_to_44',
    'male_45_to_49', 'male_50_to_54', 'male_55_to_59', 'male_60_to_61', 'male_62_to_64', 'female_under_5',
    'female_5_to_9', 'female_10_to_14', 'female_15_to_17', 'female_18_to_19', 'female_20', 'female_21',
    'female_22_to_24', 'female_25_to_29', 'female_30_to_34', 'female_35_to_39', 'female_40_to_44',
    'female_45_to_49', 'female_50_to_54', 'female_55_to_59', 'female_60_to_61', 'female_62_to_64',
    
    ## Breakdown of pop by age
    'population_1_year_and_over', 'population_3_years_over', 'pop_5_years_over', 'pop_15_and_over',
    'pop_16_over', 'pop_25_years_over', 'pop_25_64',
    
    ## Breakdown of pop by age + race
    'asian_male_45_54', 'asian_male_55_64', 'black_male_45_54', 'black_male_55_64', 'hispanic_male_45_54',
    'hispanic_male_55_64', 'white_male_45_54', 'white_male_55_64', 
    
    ## Dwellings
    'dwellings_1_units_detached', 'dwellings_1_units_attached', 'dwellings_2_units', 'dwellings_3_to_4_units', 
    'dwellings_5_to_9_units', 'dwellings_10_to_19_units', 'dwellings_20_to_49_units', 
    'dwellings_50_or_more_units', 
    
    ## Moved Houses
    'different_house_year_ago_different_city', 'different_house_year_ago_same_city',
    
    ## Degrees + male + age
    'male_45_64_associates_degree', 'male_45_64_bachelors_degree', 'male_45_64_graduate_degree',
    'male_45_64_less_than_9_grade', 'male_45_64_grade_9_12', 'male_45_64_high_school', 'male_45_64_some_college',
    'male_45_to_64',
    
    ## Type of employment
    'employed_agriculture_forestry_fishing_hunting_mining',
    'employed_arts_entertainment_recreation_accommodation_food', 'employed_construction',
    'employed_education_health_social', 'employed_finance_insurance_real_estate', 'employed_information',
    'employed_manufacturing', 'employed_other_services_not_public_admin', 'employed_public_administration',
    'employed_retail_trade', 'employed_science_management_admin_waste',
    'employed_transportation_warehousing_utilities', 'employed_wholesale_trade', 'occupation_management_arts',
    'occupation_natural_resources_construction_maintenance', 'occupation_production_transportation_material',
    'occupation_sales_office', 'occupation_services', 'sales_office_employed',
    
    ## Types of Student Populations, normalise by total_pop
    'in_grades_1_to_4', 'in_grades_5_to_8', 'in_grades_9_to_12', 'in_school', 'in_undergrad_college',
    
    ## Language spoken at home
    'speak_only_english_at_home', 'speak_spanish_at_home', 'speak_spanish_at_home_low_english',
]

### Columns to Clean

The list below contains a list of tuples in the format specified above. These are the features we want to do some manner of cleaning for (such as aggregation, normalisation, etc.)

In [ ]:
columns_to_clean = [
    ## SVI Derived Variables
    # Grouping populations into  65+ age bracket
    (
        'agg',
        [
            'female_65_to_66', 'female_67_to_69', 'female_70_to_74', 'female_75_to_79',
            'female_80_to_84', 'female_85_and_over', 'male_65_to_66', 'male_67_to_69', 
            'male_70_to_74', 'male_75_to_79', 'male_80_to_84', 'male_85_and_over'
        ],
        'aged_65_or_older',
        'total_pop'
    ),

    # Grouping population into under 17 age bracket
    (
        'agg',
        [
            'female_under_5', 'female_5_to_9', 'female_10_to_14', 'female_15_to_17', 'male_under_5',
            'male_5_to_9', 'male_10_to_14', 'male_15_to_17'
        ],
        'aged_17_or_younger',
        'total_pop'
    ),

    # Aggregating multi_unit_structures
    (
        'agg',
        [
            'dwellings_10_to_19_units', 'dwellings_20_to_49_units', 'dwellings_50_or_more_units'
        ],
        'multi_unit_structures',
        'housing_units'
    ),

    ## Normalising percent of population in poverty
    (
        'norm',
        [
            'poverty'
        ],
        'pop_determined_poverty_status'
    ),

    (
        'drop',
        [
            'pop_determined_poverty_status'
        ]
    ),

    ## Households on public assistance
    (
        'norm',
        [
            'households_public_asst_or_food_stamps'
        ],
        'households'
    ),
    
    ## Normalising Racial Populations by total_pop
    (
        'norm',
        [
            'black_pop',
            'asian_pop',
            'hispanic_pop',
            'amerindian_pop',
            'white_pop',
            'non_white_pop', 
            'other_race_pop'
        ],
        'total_pop'

    ),

    ## Dropping two_or_more_races_pop
    (
        'drop',
        [
            'two_or_more_races_pop'
        ]
    ),

    ## Population breakdown by sex
    (
        'norm',
        [
            'male_pop', 'female_pop'
        ],
        'total_pop'
    ),

    ## Marital Status
    (
        'agg',
        [
            'pop_never_married', 'pop_widowed','pop_separated', 'pop_divorced',
        ],
        'pop_currently_unmarried',
        'total_pop'
    ),

    (
        'norm',
        [
            'pop_now_married'
        ],
        'total_pop'
    ),

    ## Non-citizen population
    (
        'norm',
        [
            'not_us_citizen_pop'
        ],
        'total_pop'
    ),

    ## Normalised number of children
    (
        'norm',
        [
            'children',
        ],
        'total_pop'

    ),

    ## Household Structure
    (
        'norm',
        [
            'married_households', 'nonfamily_households', 'family_households', 
        ],
        'households'
    ),

    ## Housing Statistics
    (
        'leave',
        [
            'renter_occupied_housing_units_paying_cash_median_gross_rent',
            'owner_occupied_housing_units_lower_value_quartile', 
            'owner_occupied_housing_units_median_value',
            'owner_occupied_housing_units_upper_value_quartile'
        ]
    ),

    (
        'norm',
        [
            'occupied_housing_units', 'housing_units_renter_occupied', 'vacant_housing_units', 
            'vacant_housing_units_for_rent', 'vacant_housing_units_for_sale', 'mobile_homes',
            'multi_unit_structures'
        ],
        'housing_units'
    ),

    # Normalising popualtion living in group quarters
    (
        'norm',
        [
            'group_quarters'
        ],
        'total_pop'
    ),

    ## Housing Age
    ## Leep housing_built_1939_or_earlier AND aggregate(housing_built_2000_to_2004, 2005-later)
    (
        'norm',
        [
            'housing_built_1939_or_earlier'
        ],
        'housing_units'
    ),

    (
        'agg',
        [
            'housing_built_2005_or_later', 'housing_built_2000_to_2004'
        ],
        'housing_built_2004_or_later',
        'housing_units',
    ),
    
    ## Family Structure
    (
        'norm', 
        [
            'children_in_single_female_hh'
        ], 
        'children'
    ),

    (
        'drop', 
        [
            'two_parent_families_with_young_children',
            'one_parent_families_with_young_children'
        ]
    ),

    (
        'drop', 
        [
            'families_with_young_children',
            'father_one_parent_families_with_young_children',
            'father_in_labor_force_one_parent_families_with_young_children',
            'two_parents_in_labor_force_families_with_young_children',
            'two_parents_father_in_labor_force_families_with_young_children',
            'two_parents_mother_in_labor_force_families_with_young_children',
            'two_parents_not_in_labor_force_families_with_young_children',
        ]
    ),
    
    ## Rent burdens
    ## Aggregate into bins of <30, 30-50, and 50+ rent burden, and drop the rest/original columns
    ## Normalise with housing_units
    (
        'norm',
        [
            'rent_burden_not_computed'
        ],
        'housing_units'
    ),


    (
        'agg',
        [
            'rent_25_to_30_percent',
            'rent_20_to_25_percent', 
            'rent_15_to_20_percent',
            'rent_10_to_15_percent', 
            'rent_under_10_percent'
        ],
        'rent_under_30_percent',
        'housing_units'
    ),

    (
        'agg',
        [
            'rent_40_to_50_percent', 
            'rent_35_to_40_percent',
            'rent_30_to_35_percent'
        ],
        'rent_30_to_50_percent',
        'housing_units'

    ),

    (
        'norm',
        ['rent_over_50_percent'],
        'housing_units'
    ),

    
    ## Commute times, would need to be normalised by pop_in_labor_force
    ## ** Keep only aggregate_travel_time_to_work, walked_to_work, worked_at_home, no_car **
    (
        'norm',
        [
            'aggregate_travel_time_to_work',
            'walked_to_work',
            'worked_at_home',
            'no_car', #All people in a geographic area over the age of 16 who do not own a car
        ],
        'pop_in_labor_force'
    ),

    (
        'drop',
        [
            'commute_less_10_mins', 'commute_10_14_mins', 'commute_15_19_mins', 
            'commute_20_24_mins','commute_25_29_mins', 'commute_30_34_mins', 
            'commute_35_44_mins', 'commute_60_more_mins',
            'commute_45_59_mins', 'commuters_16_over'
        ]
    ),
    
    ## Number of cars, normalise by households
    ## ** Keep no_cars, aggreate([one_car, two_cars, three_cars, four_more_cars]) to has_cars,  
    ## normalise by households

    (
        'norm',
        [
            'no_cars'
        ],
        'households'
    ),

    (
        'agg',
        [
            'one_car', 'two_cars', 'three_cars', 'four_more_cars'
        ],
        'one_or_more_cars',
        'households'
    ),
    
    ## Commute Method
    ## normalise by pop_in_labor_force
    ## commuters_by_public_transportation dropped because it's a total and that information is already
    ## available in bus + train
    ## Keep bus, train, aggregate([car_truck_van, carpool, drove_alone])
    (
        'drop',
        [
            'commuters_by_public_transportation'
        ]
    ),
    
    (
        'norm',
        [
            'commuters_by_subway_or_elevated', 'commuters_by_bus'            
        ],
        'pop_in_labor_force'
    ),

    (
        'agg',
        [
            'commuters_by_car_truck_van', 'commuters_by_carpool', 'commuters_drove_alone'
        ],
        'commuters_personal_transport',
        'pop_in_labor_force'
    ),
    
    ## Degrees and Education, normalise by total_pop
    ## ** aggregate_into[
    ##     (high_school_including_ged, high_school_diploma),
    ##     (associates_degree, less_one_year_college, some_college_and_associates_degree),
    ##     (bachelors_degree, bachelors_degree_2),
    ##     (masters_degree, graduate_professional_degree)
    ## ]

    (
        'agg',
        [
            'less_than_high_school_graduate', 'high_school_including_ged', 'high_school_diploma'
        ],
        'education_high_school_or_less',
        'total_pop'
    ),

    (
        'agg',
        [
            'associates_degree', 'less_one_year_college', 'one_year_more_college', 
            'some_college_and_associates_degree'
        ],
        'education_less_than_bachelors',
        'total_pop'
    ),

    (
        'agg',
        [
            'bachelors_degree', 'bachelors_degree_2'
        ],
        'education_bachelors_degree',
        'total_pop'
    ),

    (
        'agg',
        [
            'masters_degree', 'graduate_professional_degree'
        ],
        'education_graduate_degree',
        'total_pop'
    ),
    
    ## Group into high/low income? Threshold? Normalise by households
    ## income categories according to US Federal Income Tax Brackets Approximations
    ## ** aggregate into categories [(0 - 14999), (15000 - 24999), (25000 - 34999), (35000 - 59999), 
    ##                              (60000 - 99999), (100000 - 199999), (200000+) 
    ##                              ]

    (
        'agg',
        [
            'income_less_10000', 'income_10000_14999',
        ],
        'income_less_14999',
        'households'
    ),

    (
        'agg',
        [
            'income_15000_19999', 'income_20000_24999'
        ],
        'income_15000_24999',
        'households'
    ),

    (
        'agg',
        [
            'income_25000_29999', 'income_30000_34999'
        ],
        'income_25000_34999',
        'households'
    ),

    (
        'agg',
        [
            'income_35000_39999', 'income_40000_44999', 'income_45000_49999', 'income_50000_59999'
        ],
        'income_35000_59999',
        'households'
    ),

    (
        'agg',
        [
            'income_60000_74999', 'income_75000_99999'
        ],
        'income_60000_99999',
        'households'
    ),

    (
        'agg',
        [
            'income_100000_124999','income_125000_149999', 'income_150000_199999'
        ],
        'income_100000_199999',
        'households'
    ),
    
    (
        'norm',
        [
            'income_200000_or_more'
        ],
        'households'
    ),

    (
        'norm',
        [
            'million_dollar_housing_units', 'mortgaged_housing_units', 'owner_occupied_housing_units'
        ],
        'housing_units'
    ),

    (
        'norm',
        [
            'employed_pop', 'unemployed_pop'
        ],
        'pop_in_labor_force'
    ),


    (
        'drop',
        [
            'management_business_sci_arts_employed',
        ]
    ),

    (
        'drop',
        [
            'not_in_labor_force', 'workers_16_and_over', 'armed_forces', 'civilian_labor_force', 
            'pop_in_labor_force', 'not_in_labor_force',
        ]
    ),
]

The code snippet below is a simple test to ensure that the elements in the `columns_to_clean` list are in the format we want. This is not an exhaustive test, however, and errors are possible.

**\[TO DO: Write more exhaustive tests?\]**

In [ ]:
assert np.all([x[0] in ['leave', 'norm', 'agg', 'drop'] for x in columns_to_clean])
assert np.all([type(x[1]) == list for x in columns_to_clean])

## Data Cleaning — Implementation

The small code snippet below first converts the `columns_to_drop` list into a `set` to ensure there are no duplicates. We then call the `clean_data` function, which implements the logic for cleaning describes above.

In [ ]:
columns_to_drop = set(columns_to_drop)
acs = clean_data(acs, columns_to_clean, columns_to_drop)

In [ ]:
list(acs.columns)

['geo_id',
 'total_pop',
 'households',
 'median_age',
 'median_income',
 'income_per_capita',
 'gini_index',
 'housing_units',
 'renter_occupied_housing_units_paying_cash_median_gross_rent',
 'owner_occupied_housing_units_lower_value_quartile',
 'owner_occupied_housing_units_median_value',
 'owner_occupied_housing_units_upper_value_quartile',
 'median_year_structure_built',
 'median_rent',
 'percent_income_spent_on_rent',
 'bachelors_degree_or_higher_25_64',
 'aged_65_or_older_normalised',
 'aged_17_or_younger_normalised',
 'multi_unit_structures_normalised',
 'poverty_normalised',
 'households_public_asst_or_food_stamps_normalised',
 'black_pop_normalised',
 'asian_pop_normalised',
 'hispanic_pop_normalised',
 'amerindian_pop_normalised',
 'white_pop_normalised',
 'non_white_pop_normalised',
 'other_race_pop_normalised',
 'male_pop_normalised',
 'female_pop_normalised',
 'pop_currently_unmarried_normalised',
 'pop_now_married_normalised',
 'not_us_citizen_pop_normalised',
 'children_

In [ ]:
# These should be 1.0 at most
acs[acs.columns[-15:]].apply(lambda x: np.max(x), axis=0)

education_less_than_bachelors_normalised    2.0
education_bachelors_degree_normalised       2.0
education_graduate_degree_normalised        2.0
income_less_14999_normalised                1.0
income_15000_24999_normalised               1.0
income_25000_34999_normalised               1.0
income_35000_59999_normalised               1.0
income_60000_99999_normalised               1.0
income_100000_199999_normalised             1.0
income_200000_or_more_normalised            1.0
million_dollar_housing_units_normalised     1.0
mortgaged_housing_units_normalised          1.0
owner_occupied_housing_units_normalised     1.0
employed_pop_normalised                     1.0
unemployed_pop_normalised                   1.0
dtype: float64

# Getting the Redlining GeoData

In [ ]:
# Reading the goe_files for each city.
geo_files = [
"/work/COVIDRedlining/data/dallas/dallas_redlined_numerical_values.geojson",
"/work/COVIDRedlining/data/detroit/detroit_redlined_numerical_values.geojson",
"/work/COVIDRedlining/data/new york/ny_redlined_numerical_values.geojson",
"/work/COVIDRedlining/data/minneapolis/minneapolis_redlined_numerical_values.geojson",
"/work/COVIDRedlining/data/california/cali_redlined_numerical_values.geojson",
"/work/COVIDRedlining/data/boston/boston_redlined_numerical_values.geojson",
"/work/COVIDRedlining/data/tampa/tampa_redlined_numerical_values.geojson",
"/work/COVIDRedlining/data/newark/newark_redlined_numerical_values.geojson",
"/work/COVIDRedlining/data/chicago/chicago_redlined_numerical_values.geojson",
"/work/COVIDRedlining/data/atlanta/atlanta_redlined_numerical_values.geojson",
]

dfs = [gpd.read_file(x) for x in geo_files]

In [ ]:
# Shows the number of rows per city
num_rows = [len(x) for x in dfs]
city_names = [x.split("_", 1)[0].split("/")[-1] for x in geo_files]
# Number of data points for each city
for x in zip(city_names, num_rows):
    print(x)

('dallas', 122)
('detroit', 573)
('ny', 2122)
('minneapolis', 124)
('cali', 2146)
('boston', 196)
('tampa', 60)
('newark', 224)
('chicago', 1033)
('atlanta', 135)


In [ ]:
# Rounding off the reweighted_redline_index to 2 decimal places to reduce spurious variability
# i.e., 3.1111 and 3.1156 are not different enough to be consider very different
geo_intersect = pd.DataFrame(pd.concat(dfs))
geo_intersect = geo_intersect[["GEOID", "reweighted_redline_index","ordinal_value"]]
geo_intersect['reweighted_redline_index'] = geo_intersect['reweighted_redline_index'].round(2)
geo_intersect

,GEOID,reweighted_redline_index,ordinal_value
0,4.811398e+10,3.10,4
1,4.811300e+10,2.09,3
2,4.811301e+10,1.00,1
3,4.811300e+10,2.16,3
4,4.811301e+10,3.00,3
...,...,...,...
130,1.312101e+10,3.00,3
131,1.312101e+10,2.46,3
132,1.312101e+10,2.00,2
133,1.308902e+10,3.00,3


In [ ]:
# A check for catching outliers with respect to the reweighted_redline_index
# np.unique(geo_intersect.reweighted_redline_index, return_counts=True)[0]
geo_intersect[geo_intersect.reweighted_redline_index < 1]

,GEOID,reweighted_redline_index,ordinal_value
258,3.608112e+10,0.16,1
1713,3.608112e+10,0.81,1
1727,3.608113e+10,0.00,1
2,6.037181e+09,0.78,1
231,6.037461e+09,0.56,1
257,6.001405e+09,0.62,1
297,6.037128e+09,0.00,1
750,6.085503e+09,0.63,1
852,6.037128e+09,0.93,1
861,6.037129e+09,0.41,1


# Joining ACS and Redlining Data

In [ ]:
# Merging the geo_files and the ACS data
joined = geo_intersect.merge(acs, how="inner", left_on="GEOID", right_on="geo_id")
joined.drop(columns=["GEOID", "ordinal_value", "geo_id"], inplace=True)

In [ ]:
# Final output to use in models
joined.to_csv("../data/ACS/acs_cleaned_data.csv", index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c9a4f701-31a5-4164-b3f3-c09cddf1309e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>